In [8]:
#This script takes every simulation with all points and calculates rmse and mae in each case

In [22]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
import matplotlib.gridspec as gridspec
import ast
import sys
sys.path.append('machine-scientist/')
sys.path.append('machine-scientist/Prior/')
from mcmc import *
from parallel import *
from fit_prior import read_prior_par
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_absolute_error

In [23]:
def clean_index(dataframe):
    dataframe.set_index('Unnamed: 0', inplace=True)
    dataframe.index.name = None
    dataframe= dataframe.reset_index(drop=True)
    return dataframe

def add_bms_pred(dataframe, bms_trace, number_param):
    VARS = ['x1',]
    x = dn[[c for c in VARS]].copy()
    y=dataframe.noise

    if number_param==10:
        prior_par = read_prior_par('machine-scientist/Prior/final_prior_param_sq.named_equations.nv1.np10.2017-10-18 18:07:35.089658.dat')
    elif number_param==20:
        prior_par = read_prior_par('machine-scientist/Prior/final_prior_param_sq.named_equations.nv1.np20.maxs200.2024-05-10 162907.551306.dat')

    #mdl model
    minrow = bms_trace[bms_trace.H == min(bms_trace.H)].iloc[0]
    minH, minexpr, minparvals = minrow.H, minrow.expr, ast.literal_eval(minrow.parvals)

    t = Tree(
        variables=list(x.columns),
        parameters=['a%d' % i for i in range(number_param)],
        x=x, y=y,
        prior_par=prior_par,
        max_size=200,
        from_string=minexpr,
    )

    t.set_par_values(deepcopy(minparvals))

    dplot = deepcopy(dn)
    dplot['ybms'] = t.predict(x)

    return dplot
    

In [31]:
#Read NN and BMS data
functions=['leaky_ReLU', 'tanh'] #tanh, leaky_ReLU
realizations=2
N=9
sigmas=[0.0, 0.02, 0.04,0.06, 0.08, 0.10, 0.12, 0.14, 0.16, 0.18, 0.20]
resolution='2x' #0.5x, 1x, 2x

if resolution=='1x':
    train_size=60
elif resolution=='0.5x':
    train_size=30
elif resolution=='2x':
    train_size=120

runid=0
NPAR=10 #10, 20
steps=50000

#Put mae and rmse of each simulation (on nn and bms) in a dataframe
for function in functions:

    for sigma in sigmas:

        for r in range(realizations+1):
            
            file_model='NN_no_overfit_' + function + '_sigma_' + str(sigma) + '_r_' + str(r) + '.csv'
            model_d='../data/' + resolution + '_resolution/trained_nns/' + file_model
            d=pd.read_csv(model_d)

            for n in range(N+1):
                #print(n)
                n_index.append(n);r_index.append(r);sigma_index.append(sigma);function_index.append(function)
            
                dn=d[d['rep']==n]
                dn=clean_index(dn)

                #Read BMS data
                if resolution=='1x':
                    filename='BMS_'+function+'_n_'+str(n)+'_sigma_'+str(sigma)+ '_r_' + str(r) + '_trace_'+str(steps)+'_prior_'+str(NPAR)+ '.csv'
                elif resolution=='0.5x':
                    filename='BMS_'+function+'_n_'+str(n)+'_sigma_'+str(sigma)+ '_r_' + str(r) + '_res_0.1_trace_'+str(steps)+'_prior_'+str(NPAR)+ '.csv'
                elif resolution=='2x':
                    filename='BMS_'+function+'_n_'+str(n)+'_sigma_'+str(sigma)+ '_r_' + str(r) + '_res_0.025_trace_'+str(steps)+'_prior_'+str(NPAR)+ '.csv'
                
                try:
                    trace=pd.read_csv('../data/MSTraces/' + resolution + '_resolution/' + filename, sep=';', header=None, names=['t', 'H', 'expr', 'parvals', 'kk1', 'kk2','kk3'])
                except pd.errors.ParserError:
                    print("faulty trace")
                    print(function, sigma, n, r)